# Servicios de atención al cliente con IA conversacional (Chatbots)

Este cuaderno acompaña el documento **"Caso de estudio: Implementación de PLN en servicios de atención al cliente con IA conversacional"**.

Objetivos del notebook:
- Visualizar la arquitectura lógica de un chatbot con PLN.
- Explorar el flujo: intención → entidades → memoria conversacional → acción de negocio.
- Probar un prototipo ejecutable con:
  - Clasificación de intenciones (zero-shot, multilingüe).
  - Extracción de entidades en español (NER).
  - Manejo de memoria / slots.
  - Integración con "APIs" simuladas.
- Observar en tiempo real la detección de intención y entidades en una interfaz Gradio.


## 1. Instalación de dependencias

Ejecuta esta celda en **Google Colab** o en un entorno con acceso a internet para instalar las librerías necesarias.


In [ ]:
# !pip install -q transformers accelerate gradio==4.40.0 torch --upgrade

## 2. Importación de librerías y carga de modelos

Se utilizan:
- Un modelo **zero-shot multilingüe** para detección de intenciones.
- Un modelo **NER en español** para extraer entidades (personas, fechas, etc.).


In [ ]:
import re
import time
import random
from typing import Dict, Any

import gradio as gr
from transformers import pipeline



In [ ]:
# Modelo zero-shot para clasificación de intenciones
intent_clf = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    device_map="auto"
)

# Modelo NER en español para extracción de entidades
ner = pipeline(
    "token-classification",
    model="PlanTL-GOB-ES/roberta-base-bne-ner",
    aggregation_strategy="simple",
    device_map="auto"
)

## 3. Dominio: intenciones y etiquetas

Se definen las intenciones clave en un escenario típico de atención al cliente:
- Consultar saldo
- Pagar factura
- Reportar falla
- Agendar cita técnica
- Saludo / Despedida
- Otro


In [ ]:
INTENTS = {
    "consultar_saldo": [
        "consultar saldo", "saldo", "cuánto debo", "mi deuda", "pagar factura", "factura"
    ],
    "pagar_factura": [
        "pagar", "pago", "cancelar deuda", "pagar factura", "realizar pago"
    ],
    "reportar_falla": [
        "falla", "no funciona", "soporte", "servicio caído", "sin señal", "reclamo"
    ],
    "agendar_cita": [
        "agendar cita", "programar visita", "técnico", "visita técnica"
    ],
    "saludo": [
        "hola", "buenos días", "buenas tardes", "buenas", "hey"
    ],
    "despedida": [
        "gracias", "adiós", "hasta luego", "nos vemos", "chao"
    ]
}

INTENT_LABELS = {
    "consultar_saldo": "Consultar saldo",
    "pagar_factura": "Pagar factura",
    "reportar_falla": "Reportar falla",
    "agendar_cita": "Agendar cita técnica",
    "saludo": "Saludo",
    "despedida": "Despedida",
    "otro": "Otro"
}

## 4. "APIs" simuladas (acciones de negocio)

Estas funciones simulan integraciones con sistemas empresariales:
- Consultar saldo
- Pagar factura
- Crear ticket de soporte
- Agendar cita técnica


In [ ]:
def api_consultar_saldo(numero_cuenta: str) -> float:
    """Simula la consulta de saldo de una cuenta."""
    random.seed(hash(numero_cuenta) % (2**32))
    return round(random.uniform(5, 120), 2)


def api_pagar_factura(numero_cuenta: str, monto: float) -> str:
    """Simula el pago de una factura."""
    time.sleep(0.6)
    comprobante = random.randint(100000, 999999)
    return f"Pago de USD {monto:.2f} aplicado a la cuenta {numero_cuenta}. Comprobante #{comprobante}."


def api_crear_ticket(nombre: str, detalle: str) -> str:
    """Simula la creación de un ticket de soporte."""
    time.sleep(0.6)
    ticket_id = random.randint(1000, 9999)
    nombre_mostrar = nombre or "cliente"
    return f"Ticket creado para {nombre_mostrar}. ID: TK-{ticket_id}. Detalle: {detalle[:60]}..."


def api_agendar_cita(nombre: str, fecha: str) -> str:
    """Simula la agenda de una cita técnica."""
    time.sleep(0.6)
    nombre_mostrar = nombre or "cliente"
    return f"Cita agendada para {nombre_mostrar} el {fecha}. Recibirás confirmación por email."

## 5. Utilidades de PLN: extracción de entidades e inferencia de intención

Incluye:
- Extracción de entidades con NER.
- Detección de números de cuenta.
- Clasificación zero-shot para decidir la intención dominante.


In [ ]:
NUMERO_CUENTA_REGEX = re.compile(r"\b(\d{8,12})\b")  # 8-12 dígitos


def extract_entities_es(texto: str) -> Dict[str, Any]:
    ents = ner(texto)
    entidades = {"PER": [], "ORG": [], "LOC": [], "MISC": [], "DATE": []}

    for e in ents:
        label = e["entity_group"]
        if label in entidades:
            entidades[label].append(e["word"])
        else:
            entidades["MISC"].append(e["word"])

    m = NUMERO_CUENTA_REGEX.search(texto)
    if m:
        entidades["ACCOUNT"] = m.group(1)

    return entidades


def guess_intent_zero_shot(texto: str) -> Dict[str, Any]:
    candidate_labels = list(INTENTS.keys())
    out = intent_clf(texto, candidate_labels, multi_label=True)
    max_label = out["labels"][0]
    max_score = float(out["scores"][0])

    if max_score < 0.35:
        return {"intent": "otro", "score": max_score}

    return {"intent": max_label, "score": max_score}

## 6. Gestor de diálogo con memoria (slots)

La "memoria" almacena información relevante de la conversación:
- Nombre del cliente.
- Número de cuenta.
- Última intención detectada.
- Último saldo consultado.


In [ ]:
def init_memory() -> Dict[str, Any]:
    return {
        "nombre": None,
        "numero_cuenta": None,
        "ultima_intencion": None,
        "ultimo_saldo": None,
    }


def fill_slots(mem: Dict[str, Any], entidades: Dict[str, Any]) -> Dict[str, Any]:
    if entidades.get("PER"):
        mem["nombre"] = mem.get("nombre") or entidades["PER"][0]
    if entidades.get("ACCOUNT"):
        mem["numero_cuenta"] = entidades["ACCOUNT"]
    return mem

## 7. Lógica principal de respuesta

En esta función se combina todo el pipeline:
1. Extraer entidades.
2. Actualizar memoria.
3. Detectar intención.
4. Ejecutar acción simulada o pedir datos faltantes.
5. Retornar mensaje del bot + contexto de depuración.


In [ ]:
def respond(user_msg: str, state: Dict[str, Any]):
    # 1) Entidades
    entidades = extract_entities_es(user_msg)

    # 2) Actualizar memoria (slots)
    state = fill_slots(state, entidades)

    # 3) Intento (intención)
    intent_out = guess_intent_zero_shot(user_msg)
    intent = intent_out["intent"]
    score = intent_out["score"]

    # 4) Flujo por intención
    if intent == "saludo":
        state["ultima_intencion"] = "saludo"
        nombre = f", {state['nombre']}" if state.get("nombre") else ""
        bot_msg = f"¡Hola{nombre}! ¿En qué puedo ayudarte hoy?"

    elif intent == "despedida":
        state["ultima_intencion"] = "despedida"
        bot_msg = "Gracias por contactarnos. ¡Que tengas un buen día!"

    elif intent == "consultar_saldo":
        state["ultima_intencion"] = "consultar_saldo"
        if not state.get("numero_cuenta"):
            bot_msg = (
                "Para consultar tu saldo, ¿me confirmas tu número de cuenta "
                "(8–12 dígitos)?"
            )
        else:
            saldo = api_consultar_saldo(state["numero_cuenta"])
            state["ultimo_saldo"] = saldo
            bot_msg = (
                f"El saldo pendiente de la cuenta {state['numero_cuenta']} es "
                f"USD {saldo:.2f}. ¿Deseas realizar el pago ahora?"
            )

    elif intent == "pagar_factura":
        state["ultima_intencion"] = "pagar_factura"
        if not state.get("numero_cuenta"):
            bot_msg = "Claro, ¿me compartes tu número de cuenta para procesar el pago?"
        else:
            saldo = state.get("ultimo_saldo")
            if saldo is None:
                saldo = api_consultar_saldo(state["numero_cuenta"])
                state["ultimo_saldo"] = saldo
            if saldo <= 0.01:
                bot_msg = (
                    f"La cuenta {state['numero_cuenta']} no presenta saldo pendiente. "
                    "¿Deseas revisar otro servicio?"
                )
            else:
                r = api_pagar_factura(state["numero_cuenta"], saldo)
                state["ultimo_saldo"] = 0.0
                bot_msg = f"{r} ¿Necesitas algo más?"

    elif intent == "reportar_falla":
        state["ultima_intencion"] = "reportar_falla"
        ticket = api_crear_ticket(state.get("nombre"), user_msg)
        bot_msg = f"He creado un ticket de soporte: {ticket}"

    elif intent == "agendar_cita":
        state["ultima_intencion"] = "agendar_cita"
        fecha = None
        for d in entidades.get("DATE", []):
            fecha = d
            break
        if not fecha:
            bot_msg = "Perfecto, ¿para qué fecha deseas agendar la visita?"
        else:
            r = api_agendar_cita(state.get("nombre"), fecha)
            bot_msg = f"{r} ¿Algo más en lo que te pueda ayudar?"

    else:
        state["ultima_intencion"] = "otro"
        bot_msg = (
            "Puedo ayudarte a consultar saldo, pagar facturas, reportar fallas "
            "o agendar citas técnicas. Cuéntame qué necesitas."
        )

    debug = {
        "intent": INTENT_LABELS.get(intent, "Otro"),
        "score": round(score, 4),
        "entities": entidades,
        "memory": state,
    }

    return bot_msg, state, debug

## 8. Interfaz interactiva con Gradio

Esta celda levanta un chatbot donde puedes escribir mensajes como:
- "Hola"
- "Quiero pagar mi factura"
- "Mi número de cuenta es 1712345678"
- "Tengo una falla en el servicio"
- "Agendar cita el 12 de agosto"

A la derecha verás un panel JSON con:
- Intención detectada
- Puntaje de confianza
- Entidades extraídas
- Estado de memoria (slots)


In [ ]:
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("## Chatbot empresarial (ES) — NLU + NER + Memoria + APIs simuladas")
    gr.Markdown(
        "Prueba con mensajes como: **'Hola'**, **'Quiero pagar mi factura'**, "
        "**'Mi número de cuenta es 1712345678'**, **'Tengo una falla en el servicio'**, "
        "**'Agendar cita el 12 de agosto'**. Observa cómo se detectan intención y entidades."
    )

    state = gr.State(init_memory())

    with gr.Row():
        chat = gr.Chatbot(label="Chat", height=360, type="messages")
        dbg = gr.JSON(label="Depuración (intención, score, entidades, memoria)")

    txt = gr.Textbox(placeholder="Escribe tu mensaje y presiona Enter...")

    def launch(user_msg, chat_history, st):
        if not user_msg:
            return chat_history, st, {}, ""
        bot_msg, st, debug = respond(user_msg, st)
        chat_history = chat_history + [
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": bot_msg},
        ]
        return chat_history, st, debug, ""

    txt.submit(launch, [txt, chat, state], [chat, state, dbg, txt])

demo.launch()

## 9. Actividades sugeridas

1. **Modificar el dominio**: añade nuevas intenciones (por ejemplo, "cambiar contraseña").
2. **Adaptar entidades**: detecta otros tipos de datos (ID de cliente, correo, etc.).
3. **Mejorar memoria**: almacena más contexto (historial de operaciones en la sesión).
4. **Conectar con APIs reales**: sustituye las funciones simuladas por servicios reales (respetando seguridad y privacidad).

Este prototipo refleja el ciclo completo descrito en el caso de estudio: comprensión, extracción, memoria e integración con sistemas externos.
